In [1]:
import json
import requests
from cobra.io.sbml import *
import pandas as pd

#############################################################################
#  Confidence Scores for rxns in generic model ##############################
#############################################################################

# read model
model = read_sbml_model('dataset/Recon2.2_formatted.xml')
print('Human Model is imported ... ')

CPLEX is available but could not load with error:
  Traceback (most recent call last):
    File "C:\Users\nadia\AppData\Roaming\Python\Python37\site-packages\optlang\__init__.py", line 43, in <module>
      from optlang import cplex_interface
    File "C:\Users\nadia\AppData\Roaming\Python\Python37\site-packages\optlang\cplex_interface.py", line 33, in <module>
      from cplex.exceptions import CplexSolverError
  ModuleNotFoundError: No module named 'cplex.exceptions'


Human Model is imported ... 


In [2]:
len(model.reactions)

8508

In [3]:
# format reaction names in Recon2
for idx,g in enumerate(model.reactions):
    if g.id == 'EX_3aib_D_b':
        new_id = 'EX_3aib__D_e'
        model.reactions[idx].id = new_id
    else:
        if g.id.startswith('EX_') and g.id.endswith('_b'):
            new_id = g.id.replace(g.id[-2:],'_e')
            model.reactions[idx].id = new_id

In [4]:
print([i.id for i in model.reactions])

['EX_10fthf_e', 'EX_11_cis_retfa_e', 'EX_13_cis_oretn_e', 'EX_13_cis_retn_e', 'EX_13_cis_retnglc_e', 'EX_1a25dhvitd3_e', 'EX_1glyc_hs_e', 'EX_1mncam_e', 'EX_23cump_e', 'EX_2425dhvitd2_e', 'EX_2425dhvitd3_e', 'EX_24nph_e', 'EX_25hvitd2_e', 'EX_25hvitd3_e', 'EX_2hb_e', 'EX_2mcit_e', 'EX_34dhoxpeg_e', 'EX_34dhphe_e', 'EX_34hpp_e', 'EX_35cgmp_e', 'EX_3aib__D_e', 'EX_3aib_e', 'EX_3bcrn_e', 'EX_3ddcrn_e', 'EX_3deccrn_e', 'EX_3hdececrn_e', 'EX_3hexdcrn_e', 'EX_3ivcrn_e', 'EX_3mlda_e', 'EX_3mob_e', 'EX_3mop_e', 'EX_3octdec2crn_e', 'EX_3octdeccrn_e', 'EX_3octdece1crn_e', 'EX_3tdcrn_e', 'EX_3tetd7ecoacrn_e', 'EX_3thexddcoacrn_e', 'EX_3ttetddcoacrn_e', 'EX_3ump_e', 'EX_4abut_e', 'EX_4abutn_e', 'EX_4hdebrisoquine_e', 'EX_4hphac_e', 'EX_4hpro_LT_e', 'EX_4mop_e', 'EX_4mptnl_e', 'EX_4mtolbutamide_e', 'EX_4nph_e', 'EX_4nphsf_e', 'EX_4pyrdx_e', 'EX_5HPET_e', 'EX_5adtststerone_e', 'EX_5adtststeroneglc_e', 'EX_5fthf_e', 'EX_5homeprazole_e', 'EX_5htrp_e', 'EX_5mta_e', 'EX_5mthf_e', 'EX_5oxpro_e', 'EX_6hts

In [6]:
# define null, true and false to avoid "NameError: name 'null' is not defined"
null = None
true = True
false = False

# store conf. scores for generic model's reactions, value is None if no conf.score is assigned
confidence_scores = {}

print('Starting database calls ...')
c=0
for idx,r in enumerate(model.reactions):
    print('Reaction ID:', r.id)
    try:
        # make a get request to BiGG models
        # bigg_response = requests.get(
        #     "http://bigg.ucsd.edu/api/v2/models/Recon3D/reactions/" + r.id)
        # bigg_json = bigg_response.content.decode("utf-8")
        # data = json.loads(bigg_json)
        # # store respective old identifier
        # old_id = data["old_identifiers"]

        bigg_response = requests.get("http://bigg.ucsd.edu/api/v2/universal/reactions/" + r.id)
        bigg_json = bigg_response.content.decode("utf-8")
        info = json.loads(bigg_json)
        # print('info:',info)
        old_id = info["old_identifiers"]

        for i in old_id:
            # call reaction from VMH using the respective old identifier
            vmh_res = requests.get(
                "https://www.vmh.life/_api/reactions/?abbreviation=" + i)

            vmh_json = vmh_res.content.decode("utf-8")
            vmh_confidence = json.loads(vmh_json)
            score = 0
            if vmh_res.status_code == 200 and len(vmh_confidence['results']):
                #print(vmh_res.status_code)

                # replace None to 0
                if vmh_confidence['results'][0]['mcs'] is None:
                    vmh_confidence['results'][0]['mcs'] = 0

                if r.id in list(confidence_scores.keys()):
                    print('ALREADY IN', r.id)

                # store confidence scores to the corresponding 'initial' identifier (found in BiGG model)
                score = vmh_confidence['results'][0]['mcs']
                #print('Confidence scores:',vmh_confidence['results'][0]['mcs'],i)
                break
          
        print(r.id, 'successfull')
        confidence_scores[r.id] = score
        c+=1

    except:
        try:
            score2 = 0
            # call reaction from VMH using the respective old identifier
            vmh_res = requests.get(
                "https://www.vmh.life/_api/reactions/?abbreviation=" + r.id)

            vmh_json = vmh_res.content.decode("utf-8")
            vmh_confidence = json.loads(vmh_json)
            if vmh_res.status_code == 200 and len(vmh_confidence['results']):

                # replace None to 0
                if vmh_confidence['results'][0]['mcs'] is None:
                    vmh_confidence['results'][0]['mcs'] = 0

                if r.id in list(confidence_scores.keys()):
                    print('ALREADY IN', r.id)

                # store confidence scores to the corresponding 'initial' identifier (found in BiGG model)
                score2 = vmh_confidence['results'][0]['mcs']
            #print('Confidence scores:',vmh_confidence['results'][0]['mcs'],i)
            print(r.id, 'successfull')
            confidence_scores[r.id] = score2
            c+=1
          
        except:
            confidence_scores[r.id] = 500
            print('rxn [%s] not found' % r.id)
            print('name:', r.name)
            print('equation:',r.reaction)
            c+=1
    
    print('-------------------')

print('Database calls are done ...')

print('Dictionary with confidence scores:', confidence_scores)
print('c',c)


Starting database calls ...
Reaction ID: EX_10fthf_e
EX_10fthf_e successfull
-------------------
Reaction ID: EX_11_cis_retfa_e
EX_11_cis_retfa_e successfull
-------------------
Reaction ID: EX_13_cis_oretn_e
EX_13_cis_oretn_e successfull
-------------------
Reaction ID: EX_13_cis_retn_e
EX_13_cis_retn_e successfull
-------------------
Reaction ID: EX_13_cis_retnglc_e
EX_13_cis_retnglc_e successfull
-------------------
Reaction ID: EX_1a25dhvitd3_e
EX_1a25dhvitd3_e successfull
-------------------
Reaction ID: EX_1glyc_hs_e
EX_1glyc_hs_e successfull
-------------------
Reaction ID: EX_1mncam_e
EX_1mncam_e successfull
-------------------
Reaction ID: EX_23cump_e
EX_23cump_e successfull
-------------------
Reaction ID: EX_2425dhvitd2_e
EX_2425dhvitd2_e successfull
-------------------
Reaction ID: EX_2425dhvitd3_e
EX_2425dhvitd3_e successfull
-------------------
Reaction ID: EX_24nph_e
EX_24nph_e successfull
-------------------
Reaction ID: EX_25hvitd2_e
EX_25hvitd2_e successfull
---------

EX_HC02204_e successfull
-------------------
Reaction ID: EX_HC02205_e
EX_HC02205_e successfull
-------------------
Reaction ID: EX_HC02206_e
EX_HC02206_e successfull
-------------------
Reaction ID: EX_HC02207_e
EX_HC02207_e successfull
-------------------
Reaction ID: EX_HC02208_e
EX_HC02208_e successfull
-------------------
Reaction ID: EX_HC02210_e
EX_HC02210_e successfull
-------------------
Reaction ID: EX_HC02213_e
EX_HC02213_e successfull
-------------------
Reaction ID: EX_HC02214_e
EX_HC02214_e successfull
-------------------
Reaction ID: EX_HC02216_e
EX_HC02216_e successfull
-------------------
Reaction ID: EX_HC02217_e
EX_HC02217_e successfull
-------------------
Reaction ID: EX_HC02220_e
EX_HC02220_e successfull
-------------------
Reaction ID: EX_Lcystin_e
EX_Lcystin_e successfull
-------------------
Reaction ID: EX_Rtotal_e
EX_Rtotal_e successfull
-------------------
Reaction ID: EX_Ser_Gly_Ala_X_Gly_e
EX_Ser_Gly_Ala_X_Gly_e successfull
-------------------
Reaction ID: E

EX_crtstrn_e successfull
-------------------
Reaction ID: EX_crvnc_e
EX_crvnc_e successfull
-------------------
Reaction ID: EX_csn_e
EX_csn_e successfull
-------------------
Reaction ID: EX_ctp_e
EX_ctp_e successfull
-------------------
Reaction ID: EX_cyan_e
EX_cyan_e successfull
-------------------
Reaction ID: EX_cynt_e
EX_cynt_e successfull
-------------------
Reaction ID: EX_cys_L_e
EX_cys_L_e successfull
-------------------
Reaction ID: EX_cysam_e
EX_cysam_e successfull
-------------------
Reaction ID: EX_cytd_e
EX_cytd_e successfull
-------------------
Reaction ID: EX_dad_2_e
EX_dad_2_e successfull
-------------------
Reaction ID: EX_dag_hs_e
EX_dag_hs_e successfull
-------------------
Reaction ID: EX_datp_e
EX_datp_e successfull
-------------------
Reaction ID: EX_dca_e
EX_dca_e successfull
-------------------
Reaction ID: EX_dchac_e
EX_dchac_e successfull
-------------------
Reaction ID: EX_dcmp_e
EX_dcmp_e successfull
-------------------
Reaction ID: EX_dcsptn1_e
EX_dcsptn1_

EX_glyc_e successfull
-------------------
Reaction ID: EX_glygly_e
EX_glygly_e successfull
-------------------
Reaction ID: EX_glygn2_e
EX_glygn2_e successfull
-------------------
Reaction ID: EX_glygn4_e
EX_glygn4_e successfull
-------------------
Reaction ID: EX_glygn5_e
EX_glygn5_e successfull
-------------------
Reaction ID: EX_glyleu_e
EX_glyleu_e successfull
-------------------
Reaction ID: EX_glyphe_e
EX_glyphe_e successfull
-------------------
Reaction ID: EX_glypro_e
EX_glypro_e successfull
-------------------
Reaction ID: EX_glysar_e
EX_glysar_e successfull
-------------------
Reaction ID: EX_gmp_e
EX_gmp_e successfull
-------------------
Reaction ID: EX_gncore2_e
EX_gncore2_e successfull
-------------------
Reaction ID: EX_gpi_sig_e
EX_gpi_sig_e successfull
-------------------
Reaction ID: EX_gsn_e
EX_gsn_e successfull
-------------------
Reaction ID: EX_gt1a_hs_e
EX_gt1a_hs_e successfull
-------------------
Reaction ID: EX_gthox_e
EX_gthox_e successfull
-------------------


EX_pheme_e successfull
-------------------
Reaction ID: EX_phyQ_e
EX_phyQ_e successfull
-------------------
Reaction ID: EX_phyt_e
EX_phyt_e successfull
-------------------
Reaction ID: EX_pi_e
EX_pi_e successfull
-------------------
Reaction ID: EX_hdca_e
EX_hdca_e successfull
-------------------
Reaction ID: EX_pmtcoa_e
EX_pmtcoa_e successfull
-------------------
Reaction ID: EX_pnto_R_e
EX_pnto_R_e successfull
-------------------
Reaction ID: EX_ppa_e
EX_ppa_e successfull
-------------------
Reaction ID: EX_ppi_e
EX_ppi_e successfull
-------------------
Reaction ID: EX_pre_prot_e
EX_pre_prot_e successfull
-------------------
Reaction ID: EX_prgstrn_e
EX_prgstrn_e successfull
-------------------
Reaction ID: EX_pro_D_e
EX_pro_D_e successfull
-------------------
Reaction ID: EX_pro_L_e
EX_pro_L_e successfull
-------------------
Reaction ID: EX_progly_e
EX_progly_e successfull
-------------------
Reaction ID: EX_prostgd2_e
EX_prostgd2_e successfull
-------------------
Reaction ID: EX_p

...

CSASULPtev successfull
-------------------
Reaction ID: CSAtd
CSAtd successfull
-------------------
Reaction ID: CSAtu
CSAtu successfull
-------------------
Reaction ID: CVM1GLUChc
CVM1GLUChc successfull
-------------------
Reaction ID: CVM23GLUChc
CVM23GLUChc successfull
-------------------
Reaction ID: CYSACMPAChc
CYSACMPAChc successfull
-------------------
Reaction ID: CYSAMPtev
CYSAMPtev successfull
-------------------
Reaction ID: DELACCRVSM23hc
DELACCRVSM23hc successfull
-------------------
Reaction ID: DEOXFVShc
DEOXFVShc successfull
-------------------
Reaction ID: DEOXFVStev
DEOXFVStev successfull
-------------------
Reaction ID: DESFVShc
DESFVShc successfull
-------------------
Reaction ID: DESFVSteb
DESFVSteb successfull
-------------------
Reaction ID: DHGLZABCt
DHGLZABCt successfull
-------------------
Reaction ID: DHGLZhc
DHGLZhc successfull
-------------------
Reaction ID: DHGLZtev
DHGLZtev successfull
-------------------
Reaction ID: DSPVSteb
DSPVSteb successfull
------

EX_atvlac_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_caribup_R_LPAREN_e_RPAREN_
EX_caribup_R_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_caribup_s_LPAREN_e_RPAREN_
EX_caribup_s_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_caribupglu_S_LPAREN_e_RPAREN_
EX_caribupglu_S_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_crglz_LPAREN_e_RPAREN_
EX_crglz_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_crvs_LPAREN_e_RPAREN_
EX_crvs_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_crvsm1_LPAREN_e_RPAREN_
EX_crvsm1_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_crvsm23_LPAREN_e_RPAREN_
EX_crvsm23_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_crvsm24_LPAREN_e_RPAREN_
EX_crvsm24_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_csa_LPAREN_e_RPAREN_
EX_csa_LPAREN_e_RPAREN_ successfull
-------------------
Reaction ID: EX_csasulp_LPAREN_e_R

LSTNM1hr successfull
-------------------
Reaction ID: LSTNM1tev
LSTNM1tev successfull
-------------------
Reaction ID: LSTNM2hr
LSTNM2hr successfull
-------------------
Reaction ID: LSTNM2tev
LSTNM2tev successfull
-------------------
Reaction ID: LSTNM4hr
LSTNM4hr successfull
-------------------
Reaction ID: LSTNM4tev
LSTNM4tev successfull
-------------------
Reaction ID: LSTNM5hr
LSTNM5hr successfull
-------------------
Reaction ID: LSTNM5tev
LSTNM5tev successfull
-------------------
Reaction ID: LSTNM7TDhc
LSTNM7TDhc successfull
-------------------
Reaction ID: LSTNM7hr
LSTNM7hr successfull
-------------------
Reaction ID: LSTNM7thc
LSTNM7thc successfull
-------------------
Reaction ID: LSTNRATt
LSTNRATt successfull
-------------------
Reaction ID: LSTNtd
LSTNtd successfull
-------------------
Reaction ID: LVACLAChep
LVACLAChep successfull
-------------------
Reaction ID: LVSTACIDhep
LVSTACIDhep successfull
-------------------
Reaction ID: LVSTACIDtu
LVSTACIDtu successfull
----------

3HPVSTETCOAitx successfull
-------------------
Reaction ID: 3HSMVitr
3HSMVitr successfull
-------------------
Reaction ID: 3ISPVSitr
3ISPVSitr successfull
-------------------
Reaction ID: 3MEACMPitr
3MEACMPitr successfull
-------------------
Reaction ID: 3OHACMPitr
3OHACMPitr successfull
-------------------
Reaction ID: 4BHGLZitr
4BHGLZitr successfull
-------------------
Reaction ID: 4HATVACIDitr
4HATVACIDitr successfull
-------------------
Reaction ID: 4HATVLACitr
4HATVLACitr successfull
-------------------
Reaction ID: 4HMDGLUCitr
4HMDGLUCitr successfull
-------------------
Reaction ID: 4OHMDZitr
4OHMDZitr successfull
-------------------
Reaction ID: 56DHPVSitr
56DHPVSitr successfull
-------------------
Reaction ID: 56EPPVSitr
56EPPVSitr successfull
-------------------
Reaction ID: 5OHFVSitr
5OHFVSitr successfull
-------------------
Reaction ID: 5OHFVSGLUitr
5OHFVSGLUitr successfull
-------------------
Reaction ID: 6AHGLZitr
6AHGLZitr successfull
-------------------
Reaction ID: 6BHG

FACOAL120i successfull
-------------------
Reaction ID: FACOAL60i
FACOAL60i successfull
-------------------
Reaction ID: HXt
HXt successfull
-------------------
Reaction ID: C60CRNt
C60CRNt successfull
-------------------
Reaction ID: C60CPT2
C60CPT2 successfull
-------------------
Reaction ID: C100CRNt
C100CRNt successfull
-------------------
Reaction ID: C100CPT2
C100CPT2 successfull
-------------------
Reaction ID: C120CRNt
C120CRNt successfull
-------------------
Reaction ID: C120CPT2
C120CPT2 successfull
-------------------
Reaction ID: AIRCr_PRASCS
AIRCr_PRASCS successfull
-------------------
Reaction ID: FAOXC10080m
FAOXC10080m successfull
-------------------
Reaction ID: FAOXC10080x
FAOXC10080x successfull
-------------------
Reaction ID: FAOXC101_3Em
FAOXC101_3Em successfull
-------------------
Reaction ID: FAOXC101_4Em
FAOXC101_4Em successfull
-------------------
Reaction ID: FAOXC101_4Zm
FAOXC101_4Zm successfull
-------------------
Reaction ID: FAOXC101_4Zx
FAOXC101_4Zx succ

In [7]:
len(confidence_scores)

8508

In [9]:
print('Saving output file ... ')
# convert dictionary to dataframe for easier storing
confscores_df = pd.DataFrame(list(confidence_scores.items()), columns=['Reaction ID', 'Confidence Score'])
confscores_df.to_csv('Recon2_confidence_scores.csv', index=False)

print('len conf scores', len(confscores_df['Confidence Score']))
print('Retrieval of confidence scores: DONE ...')

Saving output file ... 
len conf scores 8508
Retrieval of confidence scores: DONE ...
